In [1]:
from model import XLM_RoBERTa_classifier_one
from utils import get_datasets
from torch.utils.data import DataLoader
import torch
from sklearn.metrics import accuracy_score
from torch.optim import Adam
from torch import nn
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

2023-05-12 22:41:37.777467: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
#device = torch.device("cpu")
device

device(type='mps')

In [3]:
train, test, val = get_datasets()

Found cached dataset xnli (/Users/pszachew/.cache/huggingface/datasets/xnli/default-language=en/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset xnli (/Users/pszachew/.cache/huggingface/datasets/xnli/default-language=de/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset xnli (/Users/pszachew/.cache/huggingface/datasets/xnli/default-language=fr/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /Users/pszachew/.cache/huggingface/datasets/xnli/default-language=en/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd/cache-a1f3ca6fed532f45.arrow
Loading cached shuffled indices for dataset at /Users/pszachew/.cache/huggingface/datasets/xnli/default-language=en/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd/cache-c2bb5230cf3f1d09.arrow
Loading cached shuffled indices for dataset at /Users/pszachew/.cache/huggingface/datasets/xnli/default-language=en/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd/cache-53e587891365b24c.arrow


In [4]:
train_dl, test_dl, val_dl = DataLoader(train, batch_size=128, shuffle=True), DataLoader(test, batch_size=128, shuffle=True), DataLoader(val, batch_size=128, shuffle=True)

In [5]:
model = XLM_RoBERTa_classifier_one()
optimizer = Adam(model.parameters(), lr=5e-5)
model.to(device)
num_epochs = 3
loss_fun = nn.CrossEntropyLoss()

losses = []
acc = []


i = 0

for epoch in range(num_epochs):
    for batch in tqdm(train_dl):
        tok_batch = tokenizer(batch['premise'], batch['hypothesis'], return_tensors='pt', padding="max_length",
            max_length=64,
            truncation=True)
        labels = batch['label'].to(device)

        outputs = model(tok_batch['input_ids'].to(device), tok_batch['attention_mask'].to(device))
        loss = loss_fun(outputs, labels)
        acc_curr = accuracy_score(labels.cpu(), outputs.cpu().argmax(dim=1))
        acc.append(acc_curr)
        loss.backward()

        optimizer.step()
#         lr_scheduler.step()
        optimizer.zero_grad()
        # progress_bar.update(1)
        if i % 20 == 0:
            print("Train set accuracy: " + str(acc_curr))
        i = i + 1
        losses.append(loss.item())
    #torch.save(model.state_dict(), "/content/drive/MyDrive/xlm_roberta_state/xlm_roberta.pth")
    print(np.mean(losses))

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/9204 [00:00<?, ?it/s]/opt/anaconda3/envs/nlp/lib/python3.10/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py:1600: UserWarning: torch.cumsum supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/_temp/anacond

Test set accuracy: 0.328125


  0%|          | 21/9204 [00:20<2:23:06,  1.07it/s]

Test set accuracy: 0.3273809523809524


  0%|          | 41/9204 [00:39<2:25:24,  1.05it/s]

...

Test set accuracy: 0.33868769470404986


  4%|▎         | 341/9204 [06:51<3:25:27,  1.39s/it]

Test set accuracy: 0.33809109237536655


  4%|▍         | 351/9204 [07:06<2:59:09,  1.21s/it]


KeyboardInterrupt: 